In [4]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import ctrace
from tqdm import tqdm
from ctrace.simulation import *
from ctrace.utils import *

from ctrace.dataset import load_graph, load_sir_path
import concurrent.futures

In [13]:
with open('time_trails_updated.csv', "r") as csv:
    df = pd.read_csv(csv)
df.head()
df["v1_sum_values"] = df["updated_MIP"] - df["mip_value"]


In [14]:
df.head()

,Unnamed: 0,G,p,budget,from_cache,method,trial_id,min_exposed_value,mip_value,greedy_intersection,maxD,I_size,v1_size,v2_size,num_cross_edges,duration,updated_MIP,v1_sum_values
0,0,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,1,"(118.8, 12.517987058628878)",90.850904,25,28,1217,6233,11135,16931,0.883242,752.813311,661.962407
1,1,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,0,"(123.6, 9.50263121456368)",90.716560,20,28,1217,6233,11135,16931,0.885397,752.678967,661.962407
2,2,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,0,"(95.2, 14.078352176302452)",69.442667,256,28,1217,6233,11135,16931,1.148150,731.405074,661.962407
3,3,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,1,"(96.6, 11.567194992737004)",69.491244,252,28,1217,6233,11135,16931,1.222075,731.453652,661.962407
4,4,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,greedy_weighted,0,"(93.2, 11.966620241321273)",71.084567,-1,28,1217,6233,11135,16931,0.032429,733.046974,661.962407


In [10]:
min_exposed_tuples = list(map(lambda x: eval(x),df["min_exposed_value"]))

In [17]:
min_exposed_value = list(x[0] for x in min_exposed_tuples)
min_exposed_stddev = list(x[1] for x in min_exposed_tuples)

In [19]:
df["min_exposed_mean"] = min_exposed_value

In [20]:
df["min_exposed_stddev"] = min_exposed_stddev

In [23]:
df.head()

,Unnamed: 0,G,p,budget,from_cache,method,trial_id,min_exposed_value,mip_value,greedy_intersection,maxD,I_size,v1_size,v2_size,num_cross_edges,duration,updated_MIP,v1_sum_values,min_exposed_mean,min_exposed_stddev
0,0,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,1,"(118.8, 12.517987058628878)",90.850904,25,28,1217,6233,11135,16931,0.883242,752.813311,661.962407,118.8,12.517987
1,1,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,0,"(123.6, 9.50263121456368)",90.716560,20,28,1217,6233,11135,16931,0.885397,752.678967,661.962407,123.6,9.502631
2,2,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,0,"(95.2, 14.078352176302452)",69.442667,256,28,1217,6233,11135,16931,1.148150,731.405074,661.962407,95.2,14.078352
3,3,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,1,"(96.6, 11.567194992737004)",69.491244,252,28,1217,6233,11135,16931,1.222075,731.453652,661.962407,96.6,11.567195
4,4,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,greedy_weighted,0,"(93.2, 11.966620241321273)",71.084567,-1,28,1217,6233,11135,16931,0.032429,733.046974,661.962407,93.2,11.966620


In [24]:
df["updated_min_exposed_value"] = df["min_exposed_mean"] + df["v1_sum_values"]

In [18]:
with open(ctrace.PROJECT_ROOT /"output"/"run_K2VWk" / "results.csv", "r") as csv:
    df = pd.read_csv(csv)
len(df)

14000

In [19]:

G = load_graph("montgomery")

In [20]:
# Recompute the objective function

def compute_row_sum(from_cache, p=0.078):
    SIR = load_sir_path(from_cache, merge=True)
    I = SIR["I"]
    R = SIR["R"]
    v1, v2 = find_excluded_contours(G, I, R)
    P, Q = PQ_deterministic(G, I, v1, p)
    return sum(P.values())


In [28]:
df.head()

,Unnamed: 0,G,p,budget,from_cache,method,trial_id,min_exposed_value,mip_value,greedy_intersection,...,I_size,v1_size,v2_size,num_cross_edges,duration,updated_MIP,v1_sum_values,min_exposed_mean,min_exposed_stddev,updated_min_exposed_value
0,0,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,1,"(118.8, 12.517987058628878)",90.850904,25,...,1217,6233,11135,16931,0.883242,752.813311,661.962407,118.8,12.517987,780.762407
1,1,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,0,"(123.6, 9.50263121456368)",90.716560,20,...,1217,6233,11135,16931,0.885397,752.678967,661.962407,123.6,9.502631,785.562407
2,2,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,0,"(95.2, 14.078352176302452)",69.442667,256,...,1217,6233,11135,16931,1.148150,731.405074,661.962407,95.2,14.078352,757.162407
3,3,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,1,"(96.6, 11.567194992737004)",69.491244,252,...,1217,6233,11135,16931,1.222075,731.453652,661.962407,96.6,11.567195,758.562407
4,4,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,greedy_weighted,0,"(93.2, 11.966620241321273)",71.084567,-1,...,1217,6233,11135,16931,0.032429,733.046974,661.962407,93.2,11.966620,755.162407


In [22]:
cache_data = df["from_cache"]
results = []
with concurrent.futures.ProcessPoolExecutor() as executor:
    results = list(tqdm(executor.map(compute_row_sum, cache_data), total=len(cache_data)))

  0%|          | 0/14000 [00:46<?, ?it/s]

  0%|          | 0/14000 [00:47<?, ?it/s]

100%|██████████| 14000/14000 [23:40<00:00,  9.86it/s]


In [25]:
df["updated_MIP"] = results + df["mip_value"]

In [28]:
df.head(n=10)

,G,p,budget,from_cache,method,trial_id,min_exposed_value,mip_value,greedy_intersection,maxD,I_size,v1_size,v2_size,num_cross_edges,duration,updated_MIP
0,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,1,"(118.8, 12.517987058628878)",90.850904,25,28,1217,6233,11135,16931,0.883242,752.813311
1,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,0,"(123.6, 9.50263121456368)",90.716560,20,28,1217,6233,11135,16931,0.885397,752.678967
2,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,0,"(95.2, 14.078352176302452)",69.442667,256,28,1217,6233,11135,16931,1.148150,731.405074
3,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,1,"(96.6, 11.567194992737004)",69.491244,252,28,1217,6233,11135,16931,1.222075,731.453652
4,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,greedy_weighted,0,"(93.2, 11.966620241321273)",71.084567,-1,28,1217,6233,11135,16931,0.032429,733.046974
5,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,0,"(1396.6, 34.60202306224305)",526.140151,6,59,2359,25664,32967,145552,3.960133,4259.783521
6,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,0,"(94, 14.035668847618199)",68.296894,273,28,1217,6233,11135,16931,1.288606,730.259302
7,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,1,"(1399.2, 30.35951251255527)",525.913781,4,59,2359,25664,32967,145552,4.108805,4259.557151
8,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,1,"(96, 1.0)",68.296894,273,28,1217,6233,11135,16931,1.331758,730.259302
9,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,greedy_weighted,1,"(87.8, 9.391485505499118)",71.084567,-1,28,1217,6233,11135,16931,0.033301,733.046974


In [25]:
df.to_csv("time_trials_final.csv")

In [26]:
results

NameError: name 'results' is not defined

In [27]:
results

NameError: name 'results' is not defined